In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.metrics import confusion_matrix, accuracy_score
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
# training_dataset 
training = pd.read_csv("/kaggle/input/titanic/train.csv")
training.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# test_dataset
test = pd.read_csv("/kaggle/input/titanic/test.csv")
test.head()

# test.isna().sum()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
# data processing to handle the age filling 
training['Title'], test['Title'] = [df.Name.str.extract \
        (' ([A-Za-z]+)\.', expand=False) for df in [training, test]]

In [5]:
training['Age'].fillna(training.groupby('Title')['Age'].transform('mean'), inplace=True)
test['Age'].fillna(test.groupby('Title')['Age'].transform('mean'), inplace=True)

training['Age'].fillna(training['Age'].mean(),inplace=True)
test['Age'].fillna(test["Age"].mean(),inplace=True)





In [6]:
# getting the women and men who survived
women_survival = training.loc[training["Sex"]=="female"]["Survived"]
women_survival_rate_original = sum(women_survival)/len(women_survival)
print(f"The Women survival rate on the ship was {round((women_survival_rate_original*100),3)}%")

The Women survival rate on the ship was 74.204%


In [7]:
men_survival = training.loc[training["Sex"]=="male"]["Survived"]
men_survival_rate_original = sum(men_survival)/len(men_survival)
print(f"The Men survival rate on the ship was {round((men_survival_rate_original*100),3)}%")

The Men survival rate on the ship was 18.891%


In [8]:
from sklearn.ensemble import RandomForestClassifier
y = training["Survived"]
features = ["Pclass", "Sex", "SibSp", "Parch",'Age']
X = pd.get_dummies(training[features])
X_test = pd.get_dummies(test[features])
model = RandomForestClassifier(n_estimators=100, max_depth=4, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")
from sklearn.metrics import accuracy_score
# fetching the actual output
actual_df = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')
accuracy_score(actual_df['Survived'],predictions)


Your submission was successfully saved!


0.9665071770334929